# Cambiar Path absoluto a relativo para resolver path no encontrado en depth_metrics.csv

In [34]:
import csv
from pathlib import Path

# =========================
# CONFIG
# =========================
CSV_IN  = Path("Manzana/csv/depth_metrics.csv")
CSV_OUT = Path("Manzana/csv/depth_metrics_relative.csv")

# Prefijo absoluto a eliminar (Linux)
ABS_PREFIX = "/home/piper2/capturas_ros/Manzana/"

# =========================
# MAIN
# =========================
def main():
    if not CSV_IN.exists():
        raise FileNotFoundError(f"No existe: {CSV_IN}")

    CSV_OUT.parent.mkdir(parents=True, exist_ok=True)

    with open(CSV_IN, "r", newline="") as fin, open(CSV_OUT, "w", newline="") as fout:
        reader = csv.DictReader(fin)
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(fout, fieldnames=fieldnames)
        writer.writeheader()

        n = 0
        for row in reader:
            fname = row.get("filename", "")

            if fname.startswith(ABS_PREFIX):
                row["filename"] = fname[len(ABS_PREFIX):]  # path relativo

            writer.writerow(row)
            n += 1

    print(f"CSV generado correctamente: {CSV_OUT}")
    print(f"Filas procesadas: {n}")

if __name__ == "__main__":
    main()


CSV generado correctamente: Manzana\csv\depth_metrics_relative.csv
Filas procesadas: 2000


# Filtrar Imagenes 10 > 1 --output: yolo_reduced

In [35]:
import os
import re
import csv
import shutil
from pathlib import Path

# =========================
# CONFIG
# =========================
INPUT_DIR = Path("Manzana")                  # asume que ejecutas dentro de ...\Manzana
IMAGES_DIR = INPUT_DIR / "images"
CSV_DIR    = INPUT_DIR / "csv"
DEPTH_METRICS = CSV_DIR / "depth_metrics_relative.csv"

YOLO_OUT = INPUT_DIR / "yolo_reduced"
YOLO_IMAGES = YOLO_OUT / "images"
YOLO_LABELS = YOLO_OUT / "labels"

N_PER_BLOCK = 10                        # 10 tomas por tópico
DEPTH_CHOICE_METRIC = "median_z_m"      # o "center_z_m"
MIN_VALID_PX = 1
COPY_MODE = "copy"                      # "copy" o "move"

REQUIRE_FISHEYE = True                  # si True: descarta bloque si no hay fisheye cercano
# Si quieres evitar el problema por fisheye faltante (1990/2000), pon False.

# Si quieres forzar umbral, pon por ejemplo int(2e9) para 2s; por defecto None (sin límite).
MAX_STAMP_DIFF_NS = None

# =========================
# HELPERS
# =========================
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def to_float(x):
    try:
        return float(x)
    except:
        return None

def to_int(x):
    try:
        return int(float(x))
    except:
        return 0

def parse_depth_row(row):
    # header esperado: stamp_ns, filename, center_z_m, min_z_m, median_z_m, valid_px
    return {
        "stamp_ns": row.get("stamp_ns", ""),
        "filename": row.get("filename", ""),
        "center_z_m": to_float(row.get("center_z_m", "")),
        "median_z_m": to_float(row.get("median_z_m", "")),
        "valid_px": to_int(row.get("valid_px", "0")),
    }

def extract_stamp_idx(nick: str, filename: str):
    """
    Espera: <nick>_<stamp>_<idx>.png
    Devuelve: (stamp:int, idx:str)
    """
    base = os.path.basename(filename)
    m = re.match(rf"^{re.escape(nick)}_(\d+)_(\d{{4}})\.png$", base)
    if not m:
        return None, None
    return int(m.group(1)), m.group(2)

def build_index(nick: str):
    """
    idx -> list[(stamp:int, path:Path)] ordenado por stamp
    """
    folder = IMAGES_DIR / nick
    idx_map = {}
    if not folder.exists():
        return idx_map

    for p in folder.glob(f"{nick}_*_????.png"):
        stamp, idx = extract_stamp_idx(nick, p.name)
        if stamp is None:
            continue
        idx_map.setdefault(idx, []).append((stamp, p))

    for idx in idx_map:
        idx_map[idx].sort(key=lambda t: t[0])
    return idx_map

def pick_closest(candidates, target_stamp, max_diff=None):
    if not candidates:
        return None
    best = min(candidates, key=lambda t: abs(t[0] - target_stamp))
    if max_diff is not None and abs(best[0] - target_stamp) > max_diff:
        return None
    return best  # (stamp, path)

def safe_copy_or_move(src: Path, dst: Path, mode="copy"):
    ensure_dir(dst.parent)
    if mode == "move":
        shutil.move(str(src), str(dst))
    else:
        shutil.copy2(str(src), str(dst))

# =========================
# MAIN
# =========================
def main():
    if not DEPTH_METRICS.exists():
        raise FileNotFoundError(f"No existe: {DEPTH_METRICS}")

    ensure_dir(YOLO_IMAGES)
    ensure_dir(YOLO_LABELS)

    idx_color = build_index("color")
    idx_fish  = build_index("fisheye")

    # Leer depth_metrics.csv
    rows = []
    with open(DEPTH_METRICS, "r", newline="") as f:
        reader = csv.DictReader(f)
        for r in reader:
            rows.append(parse_depth_row(r))

    # Filtrar depth válidas
    rows = [r for r in rows if r["valid_px"] >= MIN_VALID_PX and r["filename"]]

    # Orden por tiempo
    def sort_key(r):
        try:
            return int(r["stamp_ns"])
        except:
            return 0
    rows.sort(key=sort_key)

    kept = []
    for i in range(0, len(rows), N_PER_BLOCK):
        block = rows[i:i+N_PER_BLOCK]
        if not block:
            continue

        metric = DEPTH_CHOICE_METRIC
        def candidate_key(r):
            v = r.get(metric, None)
            vv = v if (v is not None) else float("-inf")
            return (vv, r["valid_px"])

        best = max(block, key=candidate_key)

        # Resolver path del depth: puede venir absoluto o relativo
        depth_path = Path(best["filename"])
        if not depth_path.exists():
            depth_path_try = INPUT_DIR / depth_path
            if depth_path_try.exists():
                depth_path = depth_path_try

        stamp_depth, idx = extract_stamp_idx("depth", depth_path.name)
        if stamp_depth is None:
            print(f"[SKIP] No pude parsear depth: {depth_path.name}")
            continue

        kept.append((stamp_depth, idx, depth_path))

    print(f"Bloques procesados: {len(kept)} (objetivo: {len(kept)} tríos).")

    n_ok = 0
    n_skip = 0
    n_partial = 0

    for stamp_depth, idx, depth_path in kept:
        best_color = pick_closest(idx_color.get(idx, []), stamp_depth, MAX_STAMP_DIFF_NS)
        best_fish  = pick_closest(idx_fish.get(idx, []),  stamp_depth, MAX_STAMP_DIFF_NS)

        if best_color is None:
            # sin color no sirve para YOLO típico
            n_skip += 1
            continue
        if REQUIRE_FISHEYE and best_fish is None:
            n_skip += 1
            continue

        base_id = f"{stamp_depth}_{idx}"

        dst_depth   = YOLO_IMAGES / f"depth_{base_id}.png"
        dst_color   = YOLO_IMAGES / f"color_{base_id}.png"

        safe_copy_or_move(depth_path, dst_depth, mode=COPY_MODE)
        safe_copy_or_move(best_color[1], dst_color, mode=COPY_MODE)

        if best_fish is not None:
            dst_fisheye = YOLO_IMAGES / f"fisheye_{base_id}.png"
            safe_copy_or_move(best_fish[1], dst_fisheye, mode=COPY_MODE)
        else:
            n_partial += 1

        # Labels placeholders (si entrenas solo con color, puedes borrar depth/fisheye labels)
        for prefix in ["depth", "color", "fisheye"]:
            label_path = YOLO_LABELS / f"{prefix}_{base_id}.txt"
            if not label_path.exists():
                label_path.write_text("")

        n_ok += 1

    print("Resumen:")
    print(f"  Copiados OK (con color): {n_ok}")
    print(f"  Parciales (sin fisheye): {n_partial}")
    print(f"  Bloques descartados:     {n_skip}")
    print(f"Salida YOLO: {YOLO_OUT.resolve()}")

if __name__ == "__main__":
    main()


Bloques procesados: 200 (objetivo: 200 tríos).
Resumen:
  Copiados OK (con color): 200
  Parciales (sin fisheye): 0
  Bloques descartados:     0
Salida YOLO: C:\Users\garci\OneDrive - UNIVERSIDAD ANDRES BELLO\Desktop\1.Universidad\PhdDISA\Tesis\vision\Ultralytics\Manzana\yolo_reduced


# Desde Matlab usar export_gtruth_for_python.m para generar gTruth_py_flat.mat para trabajar con el dataset con nuevo formato

In [ ]:
export_gtruth_for_python("gTruthV1.mat","gTruth_py_flat.mat")

# Filtrar iamgenes no clases 

In [ ]:
C:/ProgramData/anaconda3/envs/yolo/python.exe filter_gtruth_flat_no_empty.py `
  --in_mat gTruth_py_flat.mat `
  --out_mat gTruth_py_flat_filtered.mat

# Crear labels en formato YOLO de gtruth.mat

In [ ]:
C:/ProgramData/anaconda3/envs/yolo/python.exe gtruth_flat_to_yolo.py `
   --mat gTruth_py_flat_filtered.mat `
   --labels_out Manzana/yolo_reduced/labels `
   --yaml_out Manzana/yolo_reduced/data.yaml `
  --dataset_root "../"

# Division de Dataset en train y val

In [40]:
from pathlib import Path
import random
import shutil
import yaml

# =========================
# CONFIG
# =========================
DATASET_DIR = Path("Manzana/yolo_reduced")
IMAGES_DIR = DATASET_DIR / "images"
LABELS_DIR = DATASET_DIR / "labels"
YAML_PATH = DATASET_DIR / "data.yaml"

# Split
VAL_RATIO = 0.2
SEED = 42

# Qué imágenes usar para entrenar (recomendado: "color_")
IMAGE_PREFIX = "color_"   # usa solo RGB
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}

# Modo: "move" o "copy"
MODE = "move"

# =========================
# HELPERS
# =========================
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def transfer(src: Path, dst: Path, mode: str):
    ensure_dir(dst.parent)
    if mode == "copy":
        shutil.copy2(src, dst)
    else:
        shutil.move(src, dst)

def main():
    if not DATASET_DIR.exists():
        raise FileNotFoundError(f"No existe DATASET_DIR: {DATASET_DIR}")

    # crear subcarpetas
    img_train = IMAGES_DIR / "train"
    img_val   = IMAGES_DIR / "val"
    lab_train = LABELS_DIR / "train"
    lab_val   = LABELS_DIR / "val"
    for p in [img_train, img_val, lab_train, lab_val]:
        ensure_dir(p)

    # buscar imágenes candidatas (solo en raíz images/, no en train/val si ya existieran)
    candidates = []
    for p in IMAGES_DIR.iterdir():
        if p.is_file() and p.suffix.lower() in IMAGE_EXTS and p.name.startswith(IMAGE_PREFIX):
            candidates.append(p)

    if not candidates:
        raise RuntimeError(
            f"No se encontraron imágenes con prefijo '{IMAGE_PREFIX}' en {IMAGES_DIR}."
        )

    # filtrar solo aquellas que tienen label correspondiente
    pairs = []
    missing = 0
    for img_path in candidates:
        label_path = LABELS_DIR / (img_path.stem + ".txt")
        if label_path.exists():
            pairs.append((img_path, label_path))
        else:
            missing += 1

    if not pairs:
        raise RuntimeError("No hay pares (imagen,label) válidos. Verifica nombres en labels/.")

    print(f"Imágenes candidatas: {len(candidates)}")
    print(f"Pares (img,label) válidos: {len(pairs)}")
    print(f"Sin label: {missing}")

    # split reproducible
    random.seed(SEED)
    random.shuffle(pairs)
    n_total = len(pairs)
    n_val = max(1, int(round(n_total * VAL_RATIO)))
    val_pairs = pairs[:n_val]
    train_pairs = pairs[n_val:]

    print(f"Train: {len(train_pairs)} | Val: {len(val_pairs)}")

    # transferir
    for img_path, lab_path in train_pairs:
        transfer(img_path, img_train / img_path.name, MODE)
        transfer(lab_path, lab_train / lab_path.name, MODE)

    for img_path, lab_path in val_pairs:
        transfer(img_path, img_val / img_path.name, MODE)
        transfer(lab_path, lab_val / lab_path.name, MODE)

    # actualizar data.yaml
    if YAML_PATH.exists():
        with open(YAML_PATH, "r", encoding="utf-8") as f:
            data = yaml.safe_load(f) or {}
    else:
        data = {}

    # En Ultralytics, lo más robusto es definir "path" y luego train/val relativos.
    data["path"] = str(DATASET_DIR).replace("\\", "/")
    data["train"] = "images/train"
    data["val"]   = "images/val"

    with open(YAML_PATH, "w", encoding="utf-8") as f:
        yaml.safe_dump(data, f, sort_keys=False, allow_unicode=True)

    print(f"OK. data.yaml actualizado: {YAML_PATH}")
    print("Estructura final esperada:")
    print(f"  {img_train}")
    print(f"  {img_val}")
    print(f"  {lab_train}")
    print(f"  {lab_val}")

if __name__ == "__main__":
    main()


Imágenes candidatas: 200
Pares (img,label) válidos: 200
Sin label: 0
Train: 160 | Val: 40
OK. data.yaml actualizado: Manzana\yolo_reduced\data.yaml
Estructura final esperada:
  Manzana\yolo_reduced\images\train
  Manzana\yolo_reduced\images\val
  Manzana\yolo_reduced\labels\train
  Manzana\yolo_reduced\labels\val


# Entrenamiento y Validacion YOLO11

In [41]:
from ultralytics import YOLO
from pathlib import Path

DATASET_YAML = Path("Manzana/yolo_reduced/data.yaml")
PROJECT_DIR  = Path("Manzana")
RUN_NAME     = "V2"

# Modelo base YOLO11 segmentation (elige tamaño: n, s, m, l, x)
# Ejemplos comunes:
#   yolo11n-seg.pt  (rápido)
#   yolo11s-seg.pt  (mejor calidad)
model = YOLO("yolo11s-seg.pt")

model.train(
    data=str(DATASET_YAML),
    imgsz=640,
    epochs=100,
    batch=8,          # ajusta según GPU/VRAM
    device=0,         # 0 si tienes GPU; "cpu" si no
    project=str(PROJECT_DIR),
    name=RUN_NAME,
    exist_ok=True,
    workers=8,
    # opcional: si tu dataset es pequeño, ayuda:
    # patience=30,
    # augment=True,
)

# Evaluación final sobre val
metrics = model.val(data=str(DATASET_YAML), device=0)
print(metrics)


New https://pypi.org/project/ultralytics/8.3.251 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.230  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Manzana\yolo_reduced\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=V2, nbs=64, nms=Fal